In [1]:
! pip install transformers datasets evaluate seqeval accelerate -U
from datasets import load_dataset,load_metric, Dataset, Features, ClassLabel
from transformers import BertForTokenClassification, BertTokenizerFast, Trainer, TrainingArguments, DataCollatorForTokenClassification
import torch
import numpy as np
import pandas as pd
import csv
import evaluate
from sklearn.metrics import confusion_matrix


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 22.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=9b102c411dc1c4a336395c72cb3e87b22dd2063f1ec60c7edc2caecc5408c993
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


2024-06-11 11:47:42.769056: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 11:47:42.769182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 11:47:42.905264: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# optional block to disable unnecessary wandb login
! pip install -q wandb
import wandb
wandb.init(mode="disabled")

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
train_data_path = '/kaggle/input/qc1a-dataset/train.tsv'
eval_data_path = '/kaggle/input/qc1a-dataset/val.tsv'
test_data_path = '/kaggle/input/qc1a-dataset/test.tsv'
save_path = '/kaggle/working/'

train_data = pd.read_csv(train_data_path, sep='\t')
eval_data = pd.read_csv(eval_data_path, sep='\t')
test_data = pd.read_csv(test_data_path, sep='\t')

def str2list(token):
    if type(token) is float:
        return []
    token = token[1:-1]
    splitted = [word[1:-1] for word in token.split(", ")]
    return splitted

train_data["tokens"] = train_data["tokens"].map(str2list)
train_data["ner_tags"] = train_data["ner_tags"].map(str2list)
eval_data["tokens"] = eval_data["tokens"].map(str2list)
eval_data["ner_tags"] = eval_data["ner_tags"].map(str2list)
test_data["tokens"] = test_data["tokens"].map(str2list)
test_data["ner_tags"] = test_data["ner_tags"].map(str2list)

train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)
test_dataset = Dataset.from_pandas(test_data)

classmap = ClassLabel(num_classes=3, names=['O', 'B-focus', 'I-focus'])

train_dataset = train_dataset.map(lambda y: {"ner_tags": classmap.str2int(y["ner_tags"])})
eval_dataset = eval_dataset.map(lambda y: {"ner_tags": classmap.str2int(y["ner_tags"])})
test_dataset = test_dataset.map(lambda y: {"ner_tags": classmap.str2int(y["ner_tags"])})

label_names = dict(zip([0, 1, 2], ['O', 'B-focus', 'I-focus']))
id2label = {i: classmap.int2str(i) for i in range(classmap.num_classes)}
label2id = {c: classmap.str2int(c) for c in classmap.names}


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/5285 [00:00<?, ? examples/s]

Map:   0%|          | 0/5285 [00:00<?, ? examples/s]

In [4]:

model_checkpoints = ["bert-base-uncased"]
for model_checkpoint in model_checkpoints:
    tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
    model = BertForTokenClassification.from_pretrained(model_checkpoint, id2label={i:classmap.int2str(i) for i in range(classmap.num_classes)},
                                                        label2id={c:classmap.str2int(c) for c in classmap.names},
                                                        finetuning_task="ner")

    def tokenize_and_align_labels(examples):
            tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
            labels = []
            for i, label in enumerate(examples["ner_tags"]):
                word_ids = tokenized_inputs.word_ids(batch_index=i)
                previous_word_idx = None
                label_ids = []
                for word_idx in word_ids:
                    if word_idx is None:
                        label_ids.append(-100)
                    elif word_idx != previous_word_idx:
                        label_ids.append(label[word_idx])
                    else:
                        label_ids.append(-100)
                    previous_word_idx = word_idx
                labels.append(label_ids)
            tokenized_inputs["labels"] = labels
            return tokenized_inputs

    train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
    eval_dataset = eval_dataset.map(tokenize_and_align_labels, batched=True)
    test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    metric = evaluate.load("seqeval")

    def compute_metrics(eval_preds):
        logits, labels = eval_preds
        predictions = np.argmax(logits, axis=-1)
        true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
        true_predictions = [
            [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

        flattened_true = [label for sublist in true_labels for label in sublist]
        flattened_pred = [label for sublist in true_predictions for label in sublist]

        labels_set = list(set(flattened_true + flattened_pred))

        cm = confusion_matrix(flattened_true, flattened_pred, labels=labels_set)

        per_class_accuracies = cm.diagonal() / cm.sum(axis=1)
        class_accuracies = {label: per_class_accuracies[idx] for idx, label in enumerate(labels_set)}

        return {
            "overall_precision": all_metrics["overall_precision"],
            "overall_recall": all_metrics["overall_recall"],
            "overall_f1": all_metrics["overall_f1"],
            "overall_accuracy": all_metrics["overall_accuracy"],
            "per_class_accuracies": class_accuracies,
            "entity_metrics": {
                entity: {
                    "precision": metrics["precision"],
                    "recall": metrics["recall"],
                    "f1": metrics["f1"],
                    "number": metrics["number"]
                }
                for entity, metrics in all_metrics.items()
                if entity not in ["overall_precision", "overall_recall", "overall_f1", "overall_accuracy"]
            }
        }

    training_args = TrainingArguments(
        output_dir=save_path,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=5e-5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    predictions, labels, _ = trainer.predict(test_dataset)
    predictions = np.argmax(predictions, axis=-1)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    errors = []
    for i, (true_label, true_prediction) in enumerate(zip(true_labels, true_predictions)):
        for j, (label, prediction) in enumerate(zip(true_label, true_prediction)):
            if label != prediction:
                errors.append({
                    "index": i,
                    "tokens": test_data.iloc[i]["tokens"],
                    "token": test_data.iloc[i]["tokens"][j],
                    "true_label": label,
                    "predicted_label": prediction
                })

    errors_df = pd.DataFrame(errors)
    errors_df.to_csv(f"{save_path}/errors.tsv", sep='\t', index=False)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/5285 [00:00<?, ? examples/s]

Map:   0%|          | 0/5285 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Per Class Accuracies,Entity Metrics
1,0.144200,0.115532,0.531523,0.549962,0.540586,0.958171,"{'O': 0.983641229281768, 'B-focus': 0.6413899020925321, 'I-focus': 0.6331676136363636}","{'focus': {'precision': 0.5315233236151603, 'recall': 0.5499622926093515, 'f1': 0.5405856189770201, 'number': 5304}}"
2,0.091500,0.126859,0.563130,0.579374,0.571137,0.959704,"{'O': 0.9823415438919583, 'B-focus': 0.676713380687272, 'I-focus': 0.6715198863636364}","{'focus': {'precision': 0.5631299248671431, 'recall': 0.5793740573152338, 'f1': 0.5711365114766287, 'number': 5304}}"
3,0.055800,0.138885,0.576119,0.582202,0.579145,0.960202,"{'O': 0.9827539901780233, 'B-focus': 0.6845843732002304, 'I-focus': 0.6701882102272727}","{'focus': {'precision': 0.5761194029850746, 'recall': 0.5822021116138764, 'f1': 0.5791447861965491, 'number': 5304}}"


Trainer is attempting to log a value of "{'O': 0.983641229281768, 'B-focus': 0.6413899020925321, 'I-focus': 0.6331676136363636}" of type <class 'dict'> for key "eval/per_class_accuracies" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'focus': {'precision': 0.5315233236151603, 'recall': 0.5499622926093515, 'f1': 0.5405856189770201, 'number': 5304}}" of type <class 'dict'> for key "eval/entity_metrics" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'O': 0.9823415438919583, 'B-focus': 0.676713380687272, 'I-focus': 0.6715198863636364}" of type <class 'dict'> for key "eval/per_class_accuracies" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'focus': {'precision': 0.5631299248671431, 

In [5]:
model.save_pretrained(save_path + "model/")
tokenizer.save_pretrained(save_path + "tokenizer/")

('/kaggle/working/tokenizer/tokenizer_config.json',
 '/kaggle/working/tokenizer/special_tokens_map.json',
 '/kaggle/working/tokenizer/vocab.txt',
 '/kaggle/working/tokenizer/added_tokens.json',
 '/kaggle/working/tokenizer/tokenizer.json')